In [8]:
import pandas as pd

df = pd.read_excel("E:/dataML-DL/final_project.ods", engine="odf", dtype= str)
df


,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,"Houston, TX","Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,"Seattle, WA","Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,"Dallas, TX",We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),"Austin, TX",Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter
...,...,...,...,...,...,...
8069,Sales Engineer,"Waltham, MA",You are the primary technical resource for the...,sales,Hardware,senior_specialist_or_project_manager
8070,"Director of Sales, ClosetMaid","Atlanta, GA",Act as the primary contact for Home Depot deci...,sales,Furniture,senior_specialist_or_project_manager
8071,PROPERTY MANAGEMENT: Regional Vice President,"Los Angeles, CA","Responsible for the management of office, mult...",finance_accounting,Real Estate Investment Companies,manager_team_leader
8072,Technical Manager SAP Senior,New York,Viseo Business Solution Practice invested in S...,consulting,IT Consulting,senior_specialist_or_project_manager


In [9]:
import re

def filter_location(location):
    result =re.findall("\,\s[A-Z]{2}$",location) 
    if len(result)!=0:
        return result [0][2:]
    else:
        return location 
    
df = df.dropna(axis=0)
df["location"] = df["location"].apply(filter_location)
df


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24156\1585409279.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["location"] = df["location"].apply(filter_location)


,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,TX,"Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,WA,"Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,TX,We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),TX,Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter
...,...,...,...,...,...,...
8069,Sales Engineer,MA,You are the primary technical resource for the...,sales,Hardware,senior_specialist_or_project_manager
8070,"Director of Sales, ClosetMaid",GA,Act as the primary contact for Home Depot deci...,sales,Furniture,senior_specialist_or_project_manager
8071,PROPERTY MANAGEMENT: Regional Vice President,CA,"Responsible for the management of office, mult...",finance_accounting,Real Estate Investment Companies,manager_team_leader
8072,Technical Manager SAP Senior,New York,Viseo Business Solution Practice invested in S...,consulting,IT Consulting,senior_specialist_or_project_manager


In [10]:
# chia data
from sklearn.model_selection import train_test_split
target = "career_level"
x = df.drop(target, axis= 1)
z = df[target]

x_train, x_test, z_train, z_test = train_test_split(x,z, test_size=0.2 , random_state= 42 )

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words= "english", ngram_range=(1,1))
result = vectorizer.fit_transform(x_train["location"])

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

processor = ColumnTransformer (transformers= [
    ("title", TfidfVectorizer(stop_words= "english", ngram_range=(1,1)),"title"),
    ("location",OneHotEncoder(handle_unknown="ignore"),["location"]),
    ("description",TfidfVectorizer(stop_words="english",ngram_range=(1,2),min_df=0.01 , max_df=0.95),"description"),
    ("function",OneHotEncoder(handle_unknown="ignore"),["function"]),
    ("industry",TfidfVectorizer(stop_words="english",ngram_range=(1,1)),"industry")
])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest,chi2
cls = Pipeline (steps= [
    ("preprocess",processor),
    ("feature_select",SelectKBest(chi2, k = 800)),
    ("model",RandomForestClassifier())
])

cls.fit(x_train,z_train)
z_pre = cls.predict(x_test)
print (classification_report(z_test , z_pre))


                                        precision    recall  f1-score   support

                        bereichsleiter       0.65      0.15      0.25       185
         director_business_unit_leader       1.00      0.19      0.32        16
                   manager_team_leader       0.64      0.74      0.68       518
managing_director_small_medium_company       0.00      0.00      0.00         1
  senior_specialist_or_project_manager       0.84      0.92      0.88       889
                            specialist       0.00      0.00      0.00         6

                              accuracy                           0.76      1615
                             macro avg       0.52      0.33      0.35      1615
                          weighted avg       0.75      0.76      0.73      1615



C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i